app.py

In [ ]:
import streamlit as st
import os
from resume_parser import extract_resume_data, extract_text
from job_matching import match_resume_to_job
from utils import save_results_to_csv, filter_results

In [ ]:
# Page configuration
st.set_page_config(page_title="ATS Resume Scanner", layout="wide")
st.title("📄 ATS Resume Scanner")

In [ ]:
# Debug info
st.write("✅ App loaded successfully!")
st.write("📝 Job description preview (if entered):")
job_description = st.text_area("📝 Paste Job Description Here", height=200)
st.write("📤 Uploaded files preview:")
uploaded_files = st.sidebar.file_uploader("📤 Upload Resumes (PDF or DOCX)", type=["pdf", "docx"], accept_multiple_files=True)

In [ ]:
if uploaded_files:
    st.write([f.name for f in uploaded_files])
else:
    st.write("No files uploaded yet.")

In [ ]:
min_score = st.sidebar.slider("🎯 Minimum Similarity Score", 0, 100, 0)
results = []

In [ ]:
# Button to analyze
if st.sidebar.button("🚀 Analyze"):
    if not uploaded_files:
        st.warning("⚠️ Please upload at least one resume file.")
    elif not job_description.strip():
        st.warning("⚠️ Please enter the job description.")
    else:
        os.makedirs("resumes", exist_ok=True)  # Ensure folder exists
        os.makedirs("output", exist_ok=True)
        
        with st.spinner("🔍 Scanning resumes..."):
            for uploaded_file in uploaded_files:
                file_path = os.path.join("resumes", uploaded_file.name)
                with open(file_path, "wb") as f:
                    f.write(uploaded_file.read())

                # Parse resume
                data = extract_resume_data(file_path)

                # 🔍 Debug print of extracted data
                st.write(f"📄 Extracted Data for {uploaded_file.name}:", data)

                name = data.get("Name", "").strip()
                email = data.get("Email", "").strip()
                phone = data.get("Phone", "").strip()
                skills = data.get("Skills", [])
                education = data.get("Education", [])
                experience = data.get("Experience", [])

                if not isinstance(skills, list):
                    skills = []
                if not isinstance(education, list):
                    education = [education] if education else []
                if not isinstance(experience, list):
                    experience = [experience] if experience else []

                resume_text = extract_text(file_path)
                score = match_resume_to_job(resume_text, job_description)

                results.append({
                    "Filename": uploaded_file.name,
                    "Name": name,
                    "Email": email,
                    "Phone": phone,
                    "Skills": ", ".join(skills),
                    "Education": ", ".join(education).strip(),
                    "Experience": ", ".join(experience).strip(),
                    "Similarity Score": score
                })

            filtered = filter_results(results, min_score)
            save_results_to_csv(filtered)

            st.success(f"✅ Processed {len(filtered)} resumes successfully.")
            st.dataframe(filtered)

            with open("output/matched_results.csv", "rb") as f:
                st.download_button("📥 Download Results CSV", data=f, file_name="matched_results.csv", mime="text/csv")

In [ ]:
def match_resume_to_job(resume_text, job_description):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    documents = [resume_text, job_description]
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return round(similarity_score * 100, 2)

In [ ]:
import os
import re
import spacy
#import docx2txt
import pdfplumber
import docx
#from pdfminer.high_level import extract_text as pdf_extract_text

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
# ---------- Extract raw text from resume ----------
# Clean up formatting
def clean_text(text):
    return '\n'.join([line.strip() for line in text.splitlines() if line.strip()])

In [ ]:
# PDF extractor using pdfplumber (accurate)
def extract_text_from_pdf(file):
    text = ""
    with pdfplumber.open(file) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return clean_text(text)

In [ ]:
# DOCX extractor
def extract_text_from_docx(file):
    doc = docx.Document(file)
    return clean_text("\n".join([para.text for para in doc.paragraphs]))

In [ ]:
def extract_text(file_path):
    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext == ".docx":
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file format. Please use PDF or DOCX.")

---------- Name Extraction with priority order ----------

In [ ]:
def normalize_name(name):
    name = re.sub(r'\s+', ' ', name.strip())
    return ' '.join([word.capitalize() for word in name.split()])

In [ ]:
def extract_candidate_name(text, filename=None):
    # Step 1: Try to extract from first few lines (visual heading)
    for line in text.strip().splitlines()[:5]:
        line_clean = line.strip()
        if line_clean and len(line_clean.split()) >= 2:
            return normalize_name(line_clean)

    # Step 2: If nothing found visually, fall back to SpaCy PERSON
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            return normalize_name(ent.text)

    # Step 3: Fallback to filename
    if filename:
        name = filename.rsplit('.', 1)[0]
        name = name.replace('_', ' ').replace('-', ' ')
        name = re.sub(r'\b(resume|cv|final|latest|profile)\b', '', name, flags=re.I)
        name = re.sub(r'\s+', ' ', name).strip()
        return normalize_name(name) if name else "Unknown"

    return "Unknown"

---------- Email Extraction ----------

In [ ]:
def extract_email(text):
    match = re.search(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', text)
    return match.group(0) if match else "Not Found"

---------- Phone Number Extraction ----------

In [ ]:
def extract_phone_number(text):
    match = re.search(r'(\+91[-\s]?)?[6-9]\d{9}', text)
    return match.group(0) if match else "Not Found"

---------- Skills Extraction ----------

In [ ]:
import re

In [ ]:
def extract_skills(text):
    # Sample skill list (can be expanded or loaded from a file)
    skill_keywords = {
        "python", "java", "c++", "sql", "mongodb", "excel", "tableau",
        "power bi", "pandas", "numpy", "seaborn", "matplotlib", "docker",
        "aws", "azure", "tensorflow", "keras", "sklearn", "git", "nlp",
        "communication", "critical thinking", "negotiation", "digital marketing"
    }

    # Normalize and lowercase text
    text = text.lower()

    found_skills = set()
    for skill in skill_keywords:
        if skill in text:
            found_skills.add(skill.title())  # Capitalize first letters

    return sorted(found_skills)

In [ ]:
def extract_education(text):
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    education = []
    current_entry = ""

    degree_keywords = [
        "bachelor", "b.tech", "b.e", "m.tech", "m.e", "master", "ph.d", "mba", "b.sc", "m.sc", "diploma", "intermediate", "secondary", "high school"
    ]
    date_pattern = re.compile(r'\d{2}/\d{4}\s*[–-]\s*\d{2}/\d{4}|\d{4}\s*[–-]\s*\d{4}')

    for i in range(len(lines)):
        line = lines[i].lower()

        if date_pattern.search(line):
            if current_entry:
                education.append(current_entry.strip())
                current_entry = ""
            current_entry = lines[i]  # Start with date line

            for j in range(1, 3):
                if i + j < len(lines):
                    next_line = lines[i + j].strip()
                    if any(k in next_line.lower() for k in degree_keywords):
                        current_entry += f" | {next_line}"

        elif any(k in line for k in degree_keywords):
            current_entry += f" | {lines[i]}"

    if current_entry:
        education.append(current_entry.strip())

    # Remove project-like entries (e.g., containing words like "prediction", "project")
    filtered = []
    for entry in education:
        if not re.search(r'(project|prediction|classification|detection)', entry, re.IGNORECASE):
            parts = entry.split(" | ")
            seen = set()
            unique = " | ".join([p for p in parts if not (p in seen or seen.add(p))])
            filtered.append(unique)

    return filtered

In [ ]:
def extract_experience(text):
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    experience = []

    date_pattern = re.compile(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|sept|oct|nov|dec)?\.?\s*\d{4}\s*[-–]\s*(present|current|\d{4})', re.IGNORECASE)

    for i in range(len(lines)):
        line = lines[i]
        if date_pattern.search(line):
            role = lines[i - 2] if i - 2 >= 0 else ""
            company = lines[i - 1] if i - 1 >= 0 else ""
            date = date_pattern.search(line).group()
            # Skip if the role line looks like a paragraph or generic description
            if len(role.split()) > 8 or "team" in role.lower():
                continue
            experience.append(f"{role} at {company} ({date})")

    return [e for e in experience if len(e.strip()) > 10 and " at " in e]

---------- Main Resume Parsing Function ----------

In [ ]:
def extract_resume_data(file_path):
    text = extract_text(file_path)
    print(text)
    name = extract_candidate_name(text, filename=file_path)
    email = extract_email(text)
    phone = extract_phone_number(text)
    skills = extract_skills(text)
    education = extract_education(text)
    experience = extract_experience(text)

    return {
        "Name": name,
        "Email": email,
        "Phone": phone,
        "Skills": skills,
        "Education": education,
        "Experience": experience,
    }